In [8]:
import pandas as pd

chunksize = 10

bams = pd.read_csv('s3://olgabot-maca/facs_bams.csv')
print(bams.shape)
bams.head()

(54912, 3)


,sample_id,bam,bai
0,A1-B000126-3_39_F-1-1_S289,s3://czbiohub-maca/remux_data/170925_A00111_00...,s3://czbiohub-maca/remux_data/170925_A00111_00...
1,A1-B000127-3_38_F-1-1_S1,s3://czbiohub-maca/remux_data/170914_A00111_00...,s3://czbiohub-maca/remux_data/170914_A00111_00...
2,A1-B000167-3_56_F-1-1_S228,s3://czbiohub-maca/remux_data/170925_A00111_00...,s3://czbiohub-maca/remux_data/170925_A00111_00...
3,A1-B000168-3_57_F-1-1_S177,s3://czbiohub-maca/remux_data/170925_A00111_00...,s3://czbiohub-maca/remux_data/170925_A00111_00...
4,A1-B000412-3_56_F-1-1_S110,s3://czbiohub-maca/remux_data/170914_A00111_00...,s3://czbiohub-maca/remux_data/170914_A00111_00...


In [5]:
! aws s3 cp s3://czbiohub-maca/remux_data/170925_A00111_0066_AH3TKNDMXX/results/D1-MAA000638-3_9_M-1-1_S159.mus.Aligned.out.sorted.bam /mnt/data/

download: s3://czbiohub-maca/remux_data/170925_A00111_0066_AH3TKNDMXX/results/D1-MAA000638-3_9_M-1-1_S159.mus.Aligned.out.sorted.bam to ../../../mnt/data/D1-MAA000638-3_9_M-1-1_S159.mus.Aligned.out.sorted.bam


In [3]:
! samtools view /mnt/data/D1-MAA000638-3_9_M-1-1_S159.mus.Aligned.out.sorted.bam | head

A00111:66:H3TKNDMXX:2:2334:28709:16767	163	chr1	15636372	255	9S78M13S	=	15636372	78	GTACATGGGGTGGTCCCGGCACTTAGCCCAATGCTGAGGCAGAAGGACTGTGAGTTTGAGGCCAGCCCAGGCAACACAATGAAATTCCTGTCTCTTATAC	FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF	NH:i:1	HI:i:1	NM:i:0	MD:Z:78
A00111:66:H3TKNDMXX:2:2462:24587:23437	99	chr1	15636372	255	9S89M2S	=	15636400	101	GTACATGGGGTGGTCCCGGCACTTAGCCCAATGCTGAGGCAGAAGGACTGTGAGTTTGAGGCCAGCCCAGGCAACACAATGAAATTCTGTCCCAAAAAAA	FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF	NH:i:1	HI:i:1	NM:i:0	MD:Z:89
A00111:66:H3TKNDMXX:2:2334:28709:16767	83	chr1	15636372	255	22S78M	=	15636372	-78	GTATAAGAGACAGGTACATGGGGTGGTCCCGGCACTTAGCCCAATGCTGAGGCAGAAGGACTGTGAGTTTGAGGCCAGCCCAGGCAACACAATGAAATTC	FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF	NH:i:1	HI:i:1	NM:i:0	MD:Z:78
A00111:66:H3TKNDMXX:2:2462:24587:23437	147	chr1	15636400	2

[AddOrReplaceReadGroups](http://broadinstitute.github.io/picard/command-line-overview.html#AddOrReplaceReadGroups)

Need to add a read group for the sample name, `RGSM`

```
java -jar picard.jar AddOrReplaceReadGroups \
      I=input.bam \
      O=output.bam \
      RGID=4 \
      RGLB=lib1 \
      RGPL=illumina \
      RGPU=unit1 \
      RGSM=20
```

Then combine multiple bam files into one with [GatherBamFiles](http://broadinstitute.github.io/picard/command-line-overview.html#GatherBamFiles)

```
java -jar picard.jar GatherBamFiles \
      I=input1.bam \ 
      I=input2.bam \ 
      O=gathered_files.bam
```

In [6]:
import os

vmax = int(np.log10(len(bams)))

for i, (sample_id, bam) in enumerate(zip(sample_ids, bams)):
    run_id = os.path.basename(bam.split('/results')[0])
    input_bam = f"/mnt/data/originals/{sample_id}.bam"
    output_bam = f"/mnt/data/merged_0/{sample_id}.bam"
    ! aws s3 cp $bam $input_bam
    ! picard AddOrReplaceReadGroups I=$input_bam O=$output_bam RGID=$sample_id RGLB=$run_id RGPL=Illumina RGPU=NovaSeq RGSM=$sample_id
    
    if i > 0:
        for n in range(1, vmax+1):
            if i %% 10**n == 0:
                previous_merged = f"/mnt/data/merged_{n-1}"
                unmerged = glob.glob(f"{previous_merged}/*.bam")
                merged = f"/mnt/data/merged_{n}/merged_{i}.bam"
                inputs = ' '.join([f'I={x}' for x in unmerged])
                ! picard GatherBamFiles $inputs O=$merged
                ! rm -rf $previous_merged/*.bam
i = n = vmax + 1
unmerged = glob.glob(f"{previous_merged}/*.bam")
merged = f"/mnt/data/merged_{n}/merged_{i}.bam"
inputs = ' '.join([f'I={x}' for x in unmerged])
! picard GatherBamFiles $inputs O=$merged

NameError: name 'sample_id' is not defined